In [ ]:
# import
import seaborn as sns
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import json
import os
import pathlib
from matplotlib.colors import ListedColormap


In [ ]:
def net1adder(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[0].replace(".pnml","")
    else:
        return 0
def net2adder(row):        
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].split('-')[1].split('.')[0].replace(".pnml","")
    else: 
        return 0
    
def pnml_remover(row):
    #get net names
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['Name'].replace(".pnml","").replace(".rdf","")
    else:
        return row['Name']

def net1NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net1'],'nNonSilentTransitions'] 
    else:
        return 0
def net2NonTaus(row):
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return df_nets.at[row['net2'],'nNonSilentTransitions'] 
    else:
        return 0
def sumNonTaus(row):    
    if row['Name'] != "Aggregated (MICRO)" and row['Name'] != "Aggregated (MACRO)":
        return row['net1NonTaus'] + row['net2NonTaus']
    else:
        return 0

def fetch_data(folder, net_analysis, dataset, profile):
    dir_list = next(os.walk(folder))[1]
    dfs = []
    #merge all 
    for subfolder in dir_list:
        #print(subfolder)
        evalFile = folder +"/" + subfolder +"/aggResults.eval"
        confFile = folder +"/" + subfolder +"/config.log"
        if os.path.exists(evalFile):
            df = pd.read_csv(evalFile ,encoding="ISO-8859-1", skipinitialspace=True)
            with open(confFile) as json_file:
                conf = json.load(json_file)
                #add config information to dataframe 
                df['matcher'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] ) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_weight'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh']) + " - Node Limit: " +str(conf['matcher']['ilp-node-limit']) + "- Time Limit : " +str(conf['matcher']['ilp-time-limit'])
                df['matcher_wo_limit'] = conf['matcher']['ilp'] + " - " + conf['matcher']['profile'] +" - " + str(conf['matcher']['word-sim']) + " - sim-weight=" + str(conf['matcher']['sim-weight']) + " - match-postprocessing=" + str(conf['matcher']['postprocessing-thresh'] )
                df['complex-matches'] = conf['matcher']['complex matches']
                df['profile'] = conf['matcher']['profile']
                df['ilp'] =  conf['matcher']['ilp']
                df['word-sim'] =  conf['matcher']['word-sim']
                df['sim-weight'] = conf['matcher']['sim-weight']
                df['matcher-postprocessing-threshold'] = conf['matcher']['postprocessing-thresh']
                df['eval-postprocessing-threshold'] = conf['evaluation']['postprocessing-thresh']
                df['ILP-time-limit'] = conf['matcher']['ilp-time-limit']
                df['ILP-node-limit'] = conf['matcher']['ilp-node-limit']
            
            #df.set_index(['Name','matcher'])
            dfs.append(df)

    df_combined = pd.concat(dfs)

    #convert time
    df_combined['OVERALL TIME'] = df_combined['OVERALL TIME'].map(lambda x: x / 1000000000.)
    df_combined['BP TIME'] = df_combined['BP TIME'].map(lambda x: x / 1000000000.)
    df_combined['LABEL-SIM TIME'] = df_combined['LABEL-SIM TIME'].map(lambda x: x / 1000000000.)
    df_combined['LP TIME'] = df_combined['LP TIME'].map(lambda x: x / 1000000000.)
    df_combined['Dataset'] = dataset
    df_combined['Prematching'] = profile

    #extend with net information stored in net_analysis
    df_nets = pd.read_csv(net_analysis ,encoding="ISO-8859-1", skipinitialspace=True)
    #df_nets.set_index('Name')


    df_combined['net1'] = df_combined.apply(lambda row: net1adder(row), axis=1)
    df_combined['net2'] = df_combined.apply(lambda row: net2adder(row), axis=1)
    #df_combined['net1NonTaus'] = df_combined.apply(lambda row: net1NonTaus(row), axis=1)
    #df_combined['net2NonTaus'] = df_combined.apply(lambda row: net2NonTaus(row), axis=1)
    #df_combined['sumNonTaus'] = df_combined.apply(lambda row: sumNonTaus(row), axis=1)
    df_combined['Name'] = df_combined.apply(lambda row: pnml_remover(row), axis=1)
    return df_combined

In [ ]:
bp_birth = "../../../eval-results/_server-results/test13/"
bp_uni = "../../../eval-results/_server-results/test17/results"
bp_sap = "../../../eval-results/_server-results/test18/results"

net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 

# load 
bp = fetch_data(bp_birth, net_birth ,"Birth", "BP")
bp = bp.append(fetch_data(bp_uni, net_uni, "Uni", "BP"))
bp = bp.append(fetch_data(bp_sap, net_sap, "Sap", "BP"))


#filter for net-net pairs only without post-processing:
bp = bp[(bp["Name"].str.contains("Aggregated") == False) & (bp["eval-postprocessing-threshold"] ==0.0)]

#define label-only dataset.
label_only = bp[bp["sim-weight"] == 0.0]
label_only = label_only[["Name","SIMILARITY"]]

#find best config per net-net pair. 
idx = bp.groupby(['Name'])['FSCORE'].transform(max) == bp['FSCORE']
bp = bp[idx]
#idx = bp.groupby(['Name'])['sim-weight'].transform(min) == bp['sim-weight']
#bp = bp[idx]

bp = bp[["Name","sim-weight","FSCORE","Dataset"]]
bp = pd.merge(bp, label_only, on='Name', how='outer')


ax = sns.regplot(x="SIMILARITY", y="sim-weight", data=bp, order=3)
ax.set_ylabel("Parameter w")
ax.set_xlabel("ILP label-only similarity")
ax.set_title("Best parameter w and label-only ILP simialrity per net-pair")

In [ ]:
sns.boxplot(x=bp["sim-weight"], y=bp["Dataset"])

In [ ]:
bp_birth = "../../../eval-results/_server-results/test13/"
bp_uni = "../../../eval-results/_server-results/test17/results"
bp_sap = "../../../eval-results/_server-results/test18/results"

net_birth = "../../../eval-results/_server-results/net-BP-birth/net.eval" 
net_uni = "../../../eval-results/_server-results/net-BP-uni/net.eval" 
net_sap = "../../../eval-results/_server-results/net-BP-sap/net.eval" 

# load 
bp = fetch_data(bp_birth, net_birth ,"Birth", "BP")
bp = bp.append(fetch_data(bp_uni, net_uni, "Uni", "BP"))
bp = bp.append(fetch_data(bp_sap, net_sap, "Sap", "BP"))


#filter for net-net pairs only without post-processing:
bp = bp[(bp["Name"].str.contains("Aggregated") == False) ]

#sns.lmplot(data=bp,x="sim-weight", y="FSCORE",order=4, hue="Dataset")

sns.regplot(data=bp,x="sim-weight", y="FSCORE",order=5)

## Result: 
- There is no clear correlation between the label-only ILP similarity and the optimal choice of parameter w. 
- 